<a href="https://colab.research.google.com/github/hank199599/data_science_from_scratch_reading_log/blob/main/Chapter22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 網路分析

* 節點
* 連線
  * 無向(undirected)連線
  * 有向(directed)連線

### 圖 22-1 DataScienceter 的朋友的關係網路
![pic 22-1](https://lh3.googleusercontent.com/oc1eCq0FSFD2iepKz6Io9t6N1oHPzVhyoqMIwMFVCjNydmcAOxyUIbOPBDFQPJI0_eVgavorbuLjkzpQDqe4x2JI6J9ZCvtFawkAf6w_c50y8k7ZqkFgelSJtKdpzSwqmwHm2D9w3f601_vJ9OSn6daRCig1f5oc87ShJcUxycp0DElkeVOYpVmC4_IW3zGncOTB6XOpG3xYJrEFz9r3pppeygEhB-llUM4OqzOPBLvnwfzrxL4qdrwO1JTbectxwzZEU0G3W41j9Ttgq08-3qaOsHTp0VUPtviP-zo9c4AE_Pw0GIh4O0GBHzVaWJlluY6DYtZ_Rxkuf1Z1m5sdUwUIebrERJu86v3mebYKx7vG9_B6m14KQyGvKEtkYS8IG8Hys3mymJmg5mm7-B4AGhHszYmcOi7XO2i0AW-P1U4veF4hrSnXD0NzQDvp7pVmMf6NOmAu9JRHW9dysSLowAiNJC9Xlh_BXB1buqJMQ551rA4nKk0yBZ3TxDA1vWrn0dNNPW4KiGMzZ55SWWduw_FX42dLkXd_JDijBe3HEVQvv6cq0XH1QLxMZlyv6O6eftIvNwKrIHjAjAUIhC7e_rEpz06fNDclvvP6qsj3oi1bZpYGwr2ovn4wGXsRXh0B-SQh9n6sOKsNC2r8-ZjLRuOSLhOxv6qEAiMAvFUJoHuN5EeK0uvxXMKDcF5dkA=w346-h111-no?authuser=0)

建構構成網路的使用者

In [15]:
users = [
  {"id":0,"name":"Hero"},
  {"id":1,"name":"Dunn"},
  {"id":2,"name":"Sue"},
  {"id":3,"name":"Chi"},
  {"id":4,"name":"Thor"},
  {"id":5,"name":"Clive"},
  {"id":6,"name":"Hicks"},
  {"id":7,"name":"Devin"},
  {"id":8,"name":"Kate"},
  {"id":9,"name":"Klein"}
     ]

以及彼此間的朋友關係

In [16]:
friendship_pairs = [
         (0,1),(0,2),(1,2),(1,3),(2,3),(3,4),
         (4,5),(5,6),(5,7),(6,8),(7,8),(8,9)
         ]

# 分支中心度 (degree centrality)
易於計算，能找出網路中擁有最多連線的節點

In [17]:
friendships = {user["id"]:[] for user in users}
for i,j in friendship_pairs:
  friendships[i].append(i)
  friendships[i].append(j)

加總每個人的朋友數量

In [18]:
def number_of_friends(user):
  """_user_ 有多少朋友?"""
  user_id = user["id"]
  friend_ids = friendships[user_id]
  return len(friend_ids)

total_connections = sum(number_of_friends(user) for user in users)

再除以使用者總人數

In [19]:
num_users = len(users)
avg_connections = total_connections/num_users

In [23]:
num_friends_by_id = [(user["id"],number_of_friends(user)) for user in users]

num_friends_by_id.sort(
    key = lambda id_and_friends:id_and_friends[1], # 根據朋友數量
    reverse = True                 # 從最多排序到最少
)
print(num_friends_by_id)

[(0, 4), (1, 4), (5, 4), (2, 2), (3, 2), (4, 2), (6, 2), (7, 2), (8, 2), (9, 0)]


依據節點的連線個數，建立列表

In [30]:
x = list(set([count[1] for count in num_friends_by_id]))
x.sort(reverse=True)
y = {number:[count[0] for count in num_friends_by_id if count[1]==number] for number in x}

print(x)
print(y)

[4, 2, 0]
{4: [0, 1, 5], 2: [2, 3, 4, 6, 7, 8], 0: [9]}


### 圖 1-2 DataSciencter的朋友關係網路，並以不同大小來呈現不同的分支之中心度
![pic 1-2](https://lh3.googleusercontent.com/46OXh6ZthDQA7Uux7sQPQc6xbZhEZZsXvCAJ37X0QAB4-p6Z6BVYuz-3B6jJbGQn3ME2A6vEneI_iakmH2aCse1m6bPs05hWtVssQWTBcS7S8mawzkuXbtm5dxihXAGgkiUmLLoWHSaZzxUK0W0BoJBs2NrZ4FFfcTzwlpcIzDfpBWxJGMOb8mbRQj8ty_41QVjukZKmIGnBlnswS3pOYddyk_By103ZBdbLpvsW-tzj-KNuytnen-1WgLDFPark8UaL73KJFggpra7-eV3quMSgiT8NIw3nollpX0OYGQdjgjbs4L8giavgvh5IEKv0k0xHzi2WWE7oMdEjsir6XUrVEw9hy-MG9I0GdY2U807aYpzhQ7S3szOvc5ltCqGeYm90ertg-ZI9QFIP6Y3dIkfM1erlBFgCgLWu26mhnSMgfr6nN0wP69SZrVEeOO94tnU0dNyLKXH5wk0NBjerKv_70pPdwae-QziOiPJeTYGDvvF6qwPyeFTk-XImQ-Y6LhiAfzrj-AuUttRJOjXNbjZKehrQxiqJHJ7v8T466bvAX9v8SdZNfze-bpebg4Jp6VCMk18Deg5yLUshWShQSnMc928LbCSCP5em-OYPpJ0IyJ5lUkVwvDuxlzJCRRjGXHMTChKEHpjdr7TcY_yi4PKblzHYMdH-Qvhivbrs8yI2Fa1DwWEy8nohklt7MQ=w332-h117-no?authuser=0)

# 居間中心度 (betweenness centrality)
找出網路中的**關鍵聯繫者**  
識別出常出現在任兩者之間最短路徑的人  
  
**例如**：欲找出節點i的居間中心度  
針對其他兩兩成對的節點j和節點k，計算出兩點之間所有最短路徑包含節點i的比例，  
最後再將各組的比例值全部加總起來


In [11]:
from typing import NamedTuple

class User(NamedTuple):
  id:int
  name:str

建構構成網路的使用者

In [32]:
users = [
      User(0,"Hero"),
      User(1,"Dunn"),
      User(2,"Sue"),
      User(3,"Chi"),
      User(4,"Thor"),
      User(5,"Clive"),
      User(6,"Hicks"),
      User(7,"Devin"),
      User(8,"Kate"),
      User(9,"Klein")
     ]

以及彼此間的朋友關係

In [35]:
friend_pairs = [
         (0,1),(0,2),(1,2),(1,3),(2,3),(3,4),
         (4,5),(5,6),(5,7),(6,8),(7,8),(8,9)
         ]

使用dict表示朋友關係

In [37]:
from typing import Dict,List

# 用來追蹤朋友關係的型別別名
Frinedships = Dict[int,List[int]]

friendships:Frinedships = {user.id:[] for user in users}

for i,j in friend_pairs:
  friendships[i].append(j)
  friendships[j].append(i)

assert friendships[4] == [3,5]
assert friendships[8] == [6,7,9]

# 建構**廣度優先搜尋演算法(BFS,Breadth-first search)**
1. 我們的目標是製作出一個以 from user為輸入項的函式,這個函式可以找出從這個 from_user 連往所有其他使用者的全部最短路徑。

2. 我們會用使用者編號組成 list 列表,來表示一條路徑。由於每條路徑都是以 from_user 做為起點,所以在列表中就不包含 from_user 的編號了。這麼一來,這個代表路徑的列表其中元素的數量,也就等於路徑本身的長度。

3. 我們會用到個叫做shortest_paths_to 的dict字典,其值為使用者編號,而相應的值則是路徑未端為該使用者的全部最短路徑列表,如果只有唯一一條最短路徑,列表中就只會包含這條路徑。如果有好幾條最短路徑,列表中則會包含所有這些最短路徑。

4. 我們也會用到一個叫做frontier的佇列(queue),裡頭會根據我們打算進行探索的順序,存放我們打算探索的使用者資料。我們是以(**前一使用者**,**使用者**)這樣的成對形式,把使用者資料存放在這個件列中如此一來,我們就可以運用前一使用者的資訊,求取出使用者相應的結果一開始我們會把from_user與他所有的朋友一一配對,然後放進這個作列中,以做為初始設定(我之前並沒有討論過「作 列」這種資料結構,它特別針對「在未端添加資料」與「從前端移出資料」這兩種 操作,做了些最佳化的設計。Python 的 collections.deque 實作了這個資料結構: 而且實作出來的其實是個雙向 [double-ended]佇列)。

5. 我們在網路中進行探索時,只要遇到還不知道其最短路徑的新相鄰節點,就把它加到佇列的末端,以便在稍後進行探索,而當前正在進行探索的使用著,就成為它的prev_user(前一使用者)。

6. 從佇列中取出某個使用者時,就算之前從沒遇過這位使用者,我們還是知道至少有一條從 from_user連往它的最短路徑,因為只要在每條連往 prev_user 的最短路徑後面再加一步就是了。

7. 從佇列中取出某個使用者時,如果我們之前遇過這位使用者,那我們所找到的新路徑,如果不是另條最短路徑(這樣的話還是要把它加進去),要不就是另一條比較長的路徑(這樣就不用再加進去了)。

8. 如果佇列中已經沒有更多使用者的資料,也就等於我們已經探索了整個網路(或者至少可以說,探索範圍已經涵蓋起始使用者可抵達的部分),而工作也就算是完成了。


In [43]:
from collections import deque

Path = List[int]

def shortest_paths_from(from_user_id:int,friendships:Frinedships) ->Dict[int,List[Path]]:
  # 一個dict字典：
  # {使用者編號:「所有」其他使用者聯網此使用者的最短編號}
  shortest_paths_to:Dict[int,List[Path]] = {from_user_id:[[]]}
  # 存放(prev_user,user)的佇列
  # 一開始將(from_user,from_user 的朋友) 全放進佇列中
  frontier = deque((from_user_id,friend_id) for friend_id in friendships[from_user_id])

  # 重複這個操作直到 frontier 佇列為空
  while frontier:
    # 自佇列前端取出下一對成對的資料
    prev_user_id,user_id = frontier.popleft()

    # 至此，已經能知道一些連往 prev_user 的最短路徑 
    paths_to_prev_user = shortest_paths_to[prev_user_id]
    new_paths_to_user = [path + [user_id] for path in paths_to_prev_user]

    # 可能已經知道一條通往 user_id 的最短路徑
    old_paths_to_user = shortest_paths_to.get(user_id,[])

    # 判定目前的路徑長度
    if old_paths_to_user:
      min_path_length = len(old_paths_to_user[0])
    else:
      min_path_length = float("inf")
    
    # 新路徑不能重複且長度需短於舊路徑
    new_paths_to_user = [path for path in new_paths_to_user if len(path) <= min_path_length and path not in old_paths_to_user]

    shortest_paths_to[user_id] = old_paths_to_user + new_paths_to_user

    # 將沒見過的節點放置到 frontier 之中
    frontier.extend((user_id,friend_id) for firend_id in friendships[user_id] if friend_id not in shortest_paths_to)
  
  return shortest_paths_to

計算所有的最短路徑

In [44]:
# 針對每個 from_user ，列出通往每個 to_user 的所有最短路徑
shortest_paths = {user_id:shortest_paths_from(user.id,friendships) for user in users}

NameError: ignored

## 計算**居間中心度**

### 圖 22-2 DataSciencter的朋友關係網路，並以不同大小來呈現不同的分支之中心度

### 圖 22-3 DataSciencter的朋友關係網路，並以不同大小來呈現不同的分支之中心度

# 本徵向量中心度 (eigenvector centrality)

## 矩陣乘法

## 中心度

## 有向圖與網頁排名